In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import seaborn 
import scipy as sp
import h5py


In [2]:
# 1. learn how to connect to the gpus 
torch.cuda.is_available()

False

In [ ]:
# run code with the gpu connected -- train.py with 500 epochs 
# and generate dataset with original numbers

In [ ]:
# learn how to store data online and access it..e.g the h5 sleap data


ls: /path/to/venv/bin/: No such file or directory


In [ ]:
# learn how to connect to the gpus remotely i.e. off campus with a VPN

/Users/briangitahi/Desktop/Summer '25/Saxena Lab/Projects/NRI/nrienv/bin/python
